In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
import random
import os
import keras
import cv2
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras

#### Open metadata file

In [ ]:
### image meta data
with h5py.File('../flame_classification/data/meta_flames_classification.h5', "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data_meta = list(f[a_group_key])

In [ ]:
data_meta.pop(0)

#### Open flame type dataset

In [ ]:
### flame type data set
with h5py.File('../flame_classification/data/y_flames_classification.h5', "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data_y = list(f[a_group_key])


In [ ]:
data_y.pop(0)

#### Open Images

In [ ]:
### image data set
with h5py.File('../flame_classification/data/x_flames_classification.h5', "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    x_data_gray = list(f[a_group_key])

In [ ]:
x_data_gray.pop(0)

In [ ]:
dataset_size = len(x_data_gray)

In [ ]:
data_y = np.asarray(data_y).astype('uint8')

In [ ]:
x_data_gray= np.asarray(x_data_gray)

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.05,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

In [ ]:
j=4
x_data_reshape = x_data_gray[j].reshape((74,74,1))
x_data_4 = x_data_reshape.reshape((1,) + x_data_reshape.shape)
name = str(data_y[j])+'_'+data_meta[j][0]+'_'+data_meta[j][1]
i = 0
for batch in datagen.flow(x_data_4, batch_size=1,
                          save_to_dir='preview', save_prefix=name, save_format='PNG'):
    i += 1
    if i > 30:
        break

In [ ]:
dt = h5py.special_dtype(vlen=str)
directory = os.fsencode('preview')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".PNG") or filename.endswith(".py"): 
        img = cv2.imread('preview/'+filename)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        image_data = np.array([img])
        flame_type_y =  np.array([int(filename.split('_')[0])])
        meta_data = np.array([['_'.join(filename.split('_')[1:-3]),filename.split('_')[-3]]],dtype = dt)
        with h5py.File('x_flames_classification.h5', 'a') as hf:
            hf["X_data"].resize((hf["X_data"].shape[0] + image_data.shape[0]), axis = 0)
            hf["X_data"][-image_data.shape[0]:] = image_data
        with h5py.File('y_flames_classification.h5', 'a') as hf1:
            hf1["Y_data"].resize((hf1["Y_data"].shape[0] + flame_type_y.shape[0]), axis = 0)
            hf1["Y_data"][-flame_type_y.shape[0]:] = flame_type_y
        with h5py.File('meta_flames_classification.h5', 'a') as hf2:
            hf2["Meta_data"].resize((hf2["Meta_data"].shape[0] + meta_data.shape[0]), axis = 0)
            hf2["Meta_data"][-meta_data.shape[0]:] = meta_data
        continue
    else:
        continue

In [ ]:
dataset_size